# ETL Medical Records

Tokenize medical records.

Template Notebook using kardiasclean.

## Part 1: Split data

1. Load data
2. Split long strings into list of strings
3. Spread list of strings into multiple rows with repeated ids (new df)

In [1]:
import pandas as pd
from pathlib import Path
from getpass import getpass

import kardiasclean
import nltk

df = pd.read_csv(Path("data_clean1.csv")).set_index("patient_id")
df.index.name = "patient_id"
df[5:15]

,gender,state,municipality,altitude,age,weight_kg,height_cm,appearance,diagnosis_general,cx_previous,diagnosis_main,date_birth,date_procedure,procedure,rachs,stay_days,expired
patient_id,,,,,,,,,,,,,,,,,
5,1,Ciudad de México,Iztapalapa,2100,2888,34.4,134.0,Normal,Ninguno,0,"Comunicación interventricular, Tipo 2 (perimem...",2004-08-31,2012-08-09,"Reparación de CIV, parche + Reparación de est...",2.0,2.0,0
6,0,Estado de México,Metepec,2620,3828,37.0,141.0,Normal,Ninguno,0,"Comunicación interventricular, Tipo 2 (perimem...",2002-04-02,2012-09-22,"Reparación de CIV, parche",2.0,2.0,0
7,0,Ciudad de México,Xochimilco,2250,1583,13.5,102.0,Normal,Ninguno,0,"Comunicación interauricular, secundum",2008-06-05,2012-09-29,"Reparacion de CIA, parche",1.0,2.0,0
8,1,Estado de México,Coacalco,2257,3549,37.0,137.0,Normal,Ninguno,0,"Comunicación interventricular, Tipo 1 (infundi...",2002-04-12,2012-10-13,"Reparación de CIV, parche",2.0,5.0,0
9,1,Estado de México,Chalco,2240,5536,62.0,162.0,Normal,VACTER,0,"Comunicación interauricular, secundum",1997-04-06,2012-10-20,"Reparación CIA, parche",1.0,2.0,0
10,1,Estado de México,Toluca,2660,4655,44.0,142.0,Normal,Trisomía 21,0,"Comunicación interventricular, Tipo 2 (perimem...",1999-11-27,2012-02-11,Reparación de CIV con parche + Reparación de e...,2.0,3.0,0
11,1,Estado de México,Tlacotepec,900,2480,12.5,160.0,Normal,Sindrome Dismorfologico E/E,0,"Comunicación interventricular, Tipo 2 (perimem...",2005-12-20,2012-10-11,Reparación de CIV con parche + Reparación de e...,2.0,6.0,0
12,1,Estado de México,San Antonio La Isla,2591,1668,14.0,98.0,Normal,Ninguno,0,"Comunicación interauricular, Secundum + Persis...",2008-03-29,2012-11-17,"Reparacion de CIA, parche + Cierre quirurgico ...",1.0,2.0,0
13,0,Ciudad de México,Xochimilco,2250,2025,17.6,114.0,Normal,Ninguno,0,"Comunicación interventricular, Tipo 1 (infundi...",2007-04-30,2012-12-15,"Reparación de CIV, cierre primario + Reparació...",2.0,2.0,0


In [2]:
df['diagnosis_general'] = kardiasclean.split_string(df['diagnosis_general'])
df['diagnosis_general']

patient_id
0                                               [Ninguno]
1                                               [Ninguno]
2                                               [Ninguno]
3                                               [Ninguno]
4                                               [Ninguno]
                              ...                        
1032                                            [Ninguno]
1033    [Síndrome cardio-facio-cutáneo, Melanocitosis ...
1034               [Sindrome Velocardiofacial FISH 22q11]
1035    [Hidrocele bilateral, Hijo de madre con diabet...
1037    [Displasia broncopulmonar, Desnutrición leve p...
Name: diagnosis_general, Length: 1003, dtype: object

In [3]:
spread_df = kardiasclean.spread_column(df['diagnosis_general'])
print(kardiasclean.get_unique_stats(spread_df))
spread_df[5:10]

                   patient_id  diagnosis_general
unique_count      1003.000000         638.000000
percent_of_total     0.702873           0.447092
avg_per_record       1.422732           2.236677


,patient_id,diagnosis_general
5,5,Ninguno
6,6,Ninguno
7,7,Ninguno
8,8,Ninguno
9,9,VACTER


## Part 2: Clean and Tokenize Strings

1. Remove accents
2. Remove Symbols with regex
3. Remove stopwords
4. Tokenize with soundex

In [4]:
spread_df['diagnosis_general'] = kardiasclean.clean_accents(spread_df['diagnosis_general'])
print(kardiasclean.get_unique_stats(spread_df))
spread_df.head()

                   patient_id  diagnosis_general
unique_count      1003.000000         628.000000
percent_of_total     0.702873           0.440084
avg_per_record       1.422732           2.272293


,patient_id,diagnosis_general
0,0,Ninguno
1,1,Ninguno
2,2,Ninguno
3,3,Ninguno
4,4,Ninguno


In [5]:
spread_df['diagnosis_general'] = kardiasclean.clean_symbols(spread_df['diagnosis_general'])
print(kardiasclean.get_unique_stats(spread_df))
spread_df.head()

                   patient_id  diagnosis_general
unique_count      1003.000000         627.000000
percent_of_total     0.702873           0.439383
avg_per_record       1.422732           2.275917


,patient_id,diagnosis_general
0,0,Ninguno
1,1,Ninguno
2,2,Ninguno
3,3,Ninguno
4,4,Ninguno


In [6]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gcaza\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
spread_df['keywords'] = kardiasclean.clean_stopwords(spread_df['diagnosis_general'])
print(kardiasclean.get_unique_stats(spread_df))
spread_df.head()

                   patient_id  diagnosis_general    keywords
unique_count      1003.000000         627.000000  615.000000
percent_of_total     0.702873           0.439383    0.430974
avg_per_record       1.422732           2.275917    2.320325


,patient_id,diagnosis_general,keywords
0,0,Ninguno,Ninguno
1,1,Ninguno,Ninguno
2,2,Ninguno,Ninguno
3,3,Ninguno,Ninguno
4,4,Ninguno,Ninguno


In [8]:
spread_df['token'] = kardiasclean.clean_tokenize(spread_df['keywords'])
print(kardiasclean.get_unique_stats(spread_df))
spread_df.head()

                   patient_id  diagnosis_general    keywords       token
unique_count      1003.000000         627.000000  615.000000  593.000000
percent_of_total     0.702873           0.439383    0.430974    0.415557
avg_per_record       1.422732           2.275917    2.320325    2.406408


,patient_id,diagnosis_general,keywords,token
0,0,Ninguno,Ninguno,NNKN
1,1,Ninguno,Ninguno,NNKN
2,2,Ninguno,Ninguno,NNKN
3,3,Ninguno,Ninguno,NNKN
4,4,Ninguno,Ninguno,NNKN


## Part 3: Get Unique List

1. Get Unique Values from the spread dataframe
2. Normalize the spread dataframe with the new unique list

In [9]:
list_df = kardiasclean.create_unique_list(spread_df, spread_df['token'])
list_df = list_df.drop(["patient_id", "index"], axis=1)
list_df.head()

,diagnosis_general,keywords,token
0,Ninguno,Ninguno,NNKN
1,VACTER,VACTER,FKTR
2,Trisomia 21,21 Trisomia,TRSM
3,Sindrome Dismorfologico E/E,Dismorfologico E/E Sindrome,TSMRFLJKSNTRM
4,Retraso Psicomotor,Psicomotor Retraso,SKMTRRTRS


In [10]:
spread_df['diagnosis_general'] = kardiasclean.normalize_from_tokens(spread_df['token'], list_df['token'], list_df['diagnosis_general'])
spread_df = spread_df.set_index("patient_id")
spread_df.tail()

,diagnosis_general,keywords,token
patient_id,,,
1035,Hidrocele bilateral,Hidrocele bilateral,HTRSLPLTRL
1035,Hijo de madre con diabetes gestacional,Hijo diabetes gestacional madre,HJTPTSJSTSNLMTR
1035,Hiperbilirrubinemia\r\nmultifactorial remitida,Hiperbilirrubinemia\r\nmultifactorial remitida,HPRPLRPNMMLTFKTRLRMTT
1037,Displasia broncopulmonar,Displasia broncopulmonar,TSPLSPRNKPLMNR
1037,Desnutricion leve por P/T,Desnutricion P/T leve,TSNTRSNPTLF


## Part 4: Store in SQL

1. NOTE: Create a database in Postgres first!
2. Rename columns if necessary.
3. Use pandas and replace, NO NEED FOR SCHEMA (CREATE TABLE ...).

In [11]:
password = getpass('Enter Database Password')
host = "kardias-test.cvj7xeynbmtt.us-east-1.rds.amazonaws.com"
pgm = kardiasclean.PostgresManager("kardias", password, host)

Enter Database Password········


In [12]:
# STORE MAIN DATA
#pgm.create_table("patients", df).count()

In [16]:
# STORE LIST DATA
#list_df = list_df.set_index("token")
pgm.create_table("diagnosis_general", list_df).count()

token                593
diagnosis_general    593
keywords             593
dtype: int64

In [18]:
# STORE SPREAD DATA
#spread_df = spread_df.drop(columns=["diagnosis_general", "keywords"])
pgm.create_table("diagnosis_general_map", spread_df).count()

patient_id    1427
token         1427
dtype: int64

## DONE!